<a href="https://colab.research.google.com/github/achrisk/Dissertation/blob/main/exp.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
!git clone https://github.com/achrisk/Dissertation.git

Cloning into 'Dissertation'...
remote: Enumerating objects: 35, done.
remote: Counting objects: 100% (35/35), done.
remote: Compressing objects: 100% (33/33), done.
remote: Total 35 (delta 3), reused 0 (delta 0), pack-reused 0
Unpacking objects: 100% (35/35), done.
Checking out files: 100% (23/23), done.


In [4]:
import scipy.io as spio
import numpy as np
import pandas as pd
from importlib.machinery import SourceFileLoader

# EEGNet-specific imports
from Dissertation.EEGModels import EEGNet, ShallowConvNet, DeepConvNet
from tensorflow.keras import utils as np_utils
from tensorflow.keras.callbacks import ModelCheckpoint
from tensorflow.keras import backend as K
K.set_image_data_format('channels_first')

from sklearn.pipeline import make_pipeline
from sklearn.linear_model import LogisticRegression

# tools for plotting confusion matrices
from matplotlib import pyplot as plt
from scipy.signal import butter, lfilter

In [5]:
# Band-pass Filter
def butter_bandpass_filter(data, lowcut, highcut, fs, order=4):
    nyq = 0.5 * fs
    low = lowcut / nyq
    high = highcut / nyq
    b, a = butter(order, [low, high], btype='band')
    y = lfilter(b, a, data)
    return y


In [ ]:
from numpy import zeros
K.clear_session()

X_tr = np.empty([80, 12, 4096])
X_ts = np.empty([40, 12, 4096])
result=[]

h_cut = [24]
drop_out = [0.25, 0.5, 0.1]
k_len = [32, 64, 128]
n_epochs = 500

outfname = 'accuray_epochs' + str(n_epochs) + '_filter_' + str(h_cut) + '_drop_' + str(drop_out) + '_patient_data.npy'

nsub = 10
nfilt = len(h_cut)
ndrop = len(drop_out)
nkl = len(k_len)
acc_sub = zeros([nsub, nfilt,ndrop,nkl])
hyper_parameter_matrix = zeros([nsub, nfilt, ndrop, nkl, 4])

for sub_idx, x in enumerate(range(1,10)):
    for h_indx, h in enumerate(h_cut):
        fName = 'Dissertation/EEG data/parsed_P0' + str(x) + 'T.mat'  # Load Data
        print(fName)
        mat = spio.loadmat(fName)
        r_X_tr = mat['RawEEGData']
        y_tr = mat['Labels']
        y_tr = y_tr.flatten() 

        print(np.shape(r_X_tr))
        print(np.shape(y_tr))

        for t in range(r_X_tr.shape[0]):
            tril = r_X_tr[t,:,:]
            #tril = tril.transpose()
            tril_filtered = butter_bandpass_filter(tril, lowcut=8, highcut=h, fs=250, order=4)
            # tril_filtered = tril_filtered.transpose()
            X_tr[t,:,:] = tril_filtered 

        # split data of each subject in training and validation
        X_train      = X_tr[0:60,:,2048:3584]
        Y_train      = y_tr[0:60]
        X_val       = X_tr[60:,:,2048:3584]
        Y_val       = y_tr[60:]

        print(np.shape(X_train))
        print(np.shape(Y_train))
        print(np.shape(X_val))
        print(np.shape(Y_val))

        # convert labels to one-hot encodings.
        Y_train = np_utils.to_categorical(Y_train-1, num_classes=2)
        Y_val = np_utils.to_categorical(Y_val-1, num_classes=2)

        kernels, chans, samples = 1, 12, 1536
        # convert data to NCHW (trials, kernels, channels, samples) format. Data 
        # contains 22 channels and 500 time-points. Set the number of kernels to 1.
        X_train      = X_train.reshape(X_train.shape[0], kernels, chans, samples)
        X_val       = X_val.reshape(X_val.shape[0], kernels, chans, samples)

        print('X_train shape:', X_train.shape)
        print(X_train.shape[0], 'train samples')
        print(X_val.shape[0], 'val samples')

        fName = 'Dissertation/EEG data/parsed_P0' + str(x) + 'E.mat'  # Load Data
        print(fName)
        mat = spio.loadmat(fName)
        r_X_ts = mat['RawEEGData']
        # y_ts = mat['Labels']
        # y_ts = y_ts.flatten() 

        print(np.shape(r_X_ts))
        # print(np.shape(y_ts))

        for t in range(r_X_ts.shape[0]):
            tril = r_X_ts[t,:,:]
            # tril = tril.transpose()
            tril_filtered = butter_bandpass_filter(tril, lowcut=8, highcut=h, fs=250, order=4)
            # tril_filtered = tril_filtered.transpose()
            X_ts[t,:,:] = tril_filtered 

        X_test      = X_ts[:,:,2048:3584]
        # Y_test      = y_ts[:]
        print(np.shape(X_test))
        # print(np.shape(Y_test))

        #convert labels to one-hot encodings.
        # Y_test      = np_utils.to_categorical(Y_test-1)

        # convert data to NCHW (trials, kernels, channels, samples) format. Data 
        # contains 22 channels and 500 time-points. Set the number of kernels to 1.
        X_test      = X_test.reshape(X_test.shape[0], kernels, chans, samples)

        print('X_train shape:', X_test.shape)
        print(X_test.shape[0], 'test samples')

        kernels, chans, samples = 1, 12, 1536
        # convert data to NCHW (trials, kernels, channels, samples) format. Data 
        # contains 22 channels and 500 time-points. Set the number of kernels to 1.
        X_train = X_train.reshape(X_train.shape[0], kernels, chans, samples)
        X_val   = X_val.reshape(X_val.shape[0], kernels, chans, samples)

        print('X_train shape:', X_train.shape)
        print(X_train.shape[0], 'train samples')
        print(X_val.shape[0], 'val samples')

        X_test      = X_ts[:,:,2048:3584]
        # Y_test      = y_ts[:]
        print(np.shape(X_test))
        # print(np.shape(Y_test))

        #convert labels to one-hot encodings.
        # Y_test      = np_utils.to_categorical(Y_test-1, num_classes=2)

        # convert data to NCHW (trials, kernels, channels, samples) format. Data 
        # contains 22 channels and 500 time-points. Set the number of kernels to 1.
        X_test      = X_test.reshape(X_test.shape[0], kernels, chans, samples)

        print('X_train shape:', X_test.shape)
        print(X_test.shape[0], 'train samples')

        # acc_sub = zeros([rows, cols])
        for id_d, d in enumerate(drop_out):
            for id_kl, kl in enumerate(k_len):
              print(id_kl, id_d)
              # configure the EEGNet-8,2,16 model with kernel length of 32 samples (other 
              # model configurations may do better, but this is a good starting point)
              model = EEGNet(nb_classes = 2, Chans = 12, Samples = 1536,
                             dropoutRate = d, kernLength = kl, F1 = 8,D = 2, F2 = 16,
                             norm_rate = 0.25, dropoutType = 'Dropout')

              # compile the model and set the optimizers
              model.compile(loss='categorical_crossentropy', optimizer='adam', 
                            metrics = ['accuracy'])

              # count number of parameters in the model
              numParams    = model.count_params() 

              # set a valid path for your system to record model checkpoints
              checkpointer = ModelCheckpoint(filepath='/tmp/checkpoint.h5', verbose=1, save_best_only=True)

              # the syntax is {class_1:weight_1, class_2:weight_2,...}. Here just setting
              # the weights all to be 1
              class_weights = {0:1, 1:1}

              history = model.fit(X_train, Y_train, batch_size = 16, epochs = n_epochs, 
                                  verbose = 2, validation_data=(X_val, Y_val),
                                  callbacks=[checkpointer], class_weight = class_weights)

              print('\n# Evaluate on test data')


              results = model.evaluate(X_val, Y_val)
              # TODO(achristofi): When you get proper validation labels change this back to X_test and Y_test
              print('test loss, test acc:', results)

              print(results)
              acc_sub[sub_idx,h_indx,id_d, id_kl] = results[1]
              hyper_parameter_matrix[sub_idx,h_indx,id_d, id_kl, :] = [x,h,d, kl]

              from keras import backend as K 
              # Do some code, e.g. train and save model
              K.clear_session()
              
np.save(outfname, acc_sub)


Streaming output truncated to the last 5000 lines.

Epoch 00122: val_loss did not improve from 0.73849
4/4 - 0s - loss: 0.1890 - accuracy: 0.9000 - val_loss: 2.1305 - val_accuracy: 0.3500
Epoch 123/500

Epoch 00123: val_loss did not improve from 0.73849
4/4 - 0s - loss: 0.1470 - accuracy: 0.9667 - val_loss: 2.0220 - val_accuracy: 0.3500
Epoch 124/500

Epoch 00124: val_loss did not improve from 0.73849
4/4 - 0s - loss: 0.1212 - accuracy: 0.9833 - val_loss: 2.0885 - val_accuracy: 0.3500
Epoch 125/500

Epoch 00125: val_loss did not improve from 0.73849
4/4 - 0s - loss: 0.1293 - accuracy: 0.9500 - val_loss: 2.0994 - val_accuracy: 0.4000
Epoch 126/500

Epoch 00126: val_loss did not improve from 0.73849
4/4 - 0s - loss: 0.1125 - accuracy: 1.0000 - val_loss: 2.0824 - val_accuracy: 0.4000
Epoch 127/500

Epoch 00127: val_loss did not improve from 0.73849
4/4 - 0s - loss: 0.1731 - accuracy: 0.9667 - val_loss: 2.1251 - val_accuracy: 0.4000
Epoch 128/500

Epoch 00128: val_loss did not improve from

In [ ]:
print(results)

[2.820241928100586, 0.6000000238418579]


In [ ]:
ind = np.unravel_index(np.argmax(acc_sub, axis=None), acc_sub.shape)
acc_sub
hyper_parameter_matrix
hyper_parameter_matrix[ind]
reduced_acc_sub = np.mean(acc_sub, axis=0, keepdims=True)
ind_reduced = np.unravel_index(np.argmax(reduced_acc_sub, axis=None), reduced_acc_sub.shape)
print('Index is: {}'.format(ind_reduced[-1]))
print('Hyperparameters are: {}'.format(hyper_parameter_matrix[ind_reduced]))
print(hyper_parameter_matrix[ind_reduced])
print(acc_sub)

Index is: 0
Hyperparameters are: [ 1.   24.    0.25 32.  ]
[ 1.   24.    0.25 32.  ]
[[[[0.80000001 0.60000002 0.64999998]]]


 [[[0.75       0.75       0.75      ]]]


 [[[0.05       0.         0.        ]]]


 [[[0.89999998 0.94999999 0.75      ]]]


 [[[0.30000001 0.40000001 0.30000001]]]


 [[[0.55000001 0.44999999 0.5       ]]]


 [[[0.5        0.5        0.5       ]]]


 [[[0.80000001 0.64999998 0.60000002]]]


 [[[0.         0.         0.        ]]]


 [[[0.         0.         0.        ]]]]
